# Normailty Check of Electrochemical Cell Connectivity:

## Intitialization

In [ ]:
import os
import time

from workflow_config import *
from acl_pyro_objects import ACL_Pyro_Client
from biologic_sp200_config import SP200_config_params, SP200_Technique_params
from jkem_setup_config import Serial_Port_MetaData_Dic
from ml_models_for_normality_check import *

## I-V Profile Normality Check Learning: Feature Design and Extraction

In [ ]:
save_file_flag=False
show_IV_plot=True
# Learn the model and preform feature extraction
i_probe_lst=GPR_for_CV_feature_extraction(training_data_path,v_probe,save_file_flag,show_IV_plot) # 1-2
# Assign Classes to the training data set
i_probe_lst=call_assign_classes(i_probe_lst) # 1-3
# Train a classifer with the probing measurement
call_Train_n_Serialize_RF_Classifier(i_probe_lst,EoT_Classifier_Path, EoT_Classifier) #1-4

## Trigger Electrochemistry Workflow

In [ ]:
PS200_WF=ACL_Pyro_Client(ipAddressServer,connectionPort)

## Initialize & Run  J-Kem Electrochemistry Setup

In [ ]:
sleep_time=3
s=time.time()
PS200_WF.call_Initialize_JKem_API(Serial_Port_MetaData_Dic)
time.sleep(sleep_time)
PS200_WF.call_Activate_JKem_API()
time.sleep(sleep_time)
time.sleep(sleep_time)
PS200_WF.Set_Rate_SyringePump(1,10.0) # (syringepump id, rate)
time.sleep(sleep_time)
# port 8 is for Fraction Collector
PS200_WF.Set_Port_SyringePump(1,8)  # (syringepump id, syringepump port)
time.sleep(sleep_time)
PS200_WF.FractionCollector_Rack('20ml')
time.sleep(sleep_time)
PS200_WF.Set_Vial_FractionCollector(1,'BOTTOM') # (vial id, position)
time.sleep(sleep_time)
PS200_WF.Withdraw_SyringePump(1,7.0,10.0) # (syringepump id, w_volume, rate)
time.sleep(sleep_time)
# port 2 is for cell
PS200_WF.Set_Port_SyringePump(1,2)  # (syringepump id, syringepump port)
time.sleep(sleep_time)

PS200_WF.Dispense_SyringePump(1,7.0,10.0) # (syringepump id, d_volume, rate)
time.sleep(sleep_time)
# iport 3 is for Air
PS200_WF.Set_Port_SyringePump(1,3)  # (syringepump id, syringepump port)
time.sleep(sleep_time)
PS200_WF.Withdraw_SyringePump(1,5.0,10.0) # (syringepump id, w_volume, rate)
time.sleep(sleep_time)
PS200_WF.Set_Port_SyringePump(1,2)  # (syringepump id, syringepump port)
time.sleep(sleep_time)
PS200_WF.Dispense_SyringePump(1,5.0,10.0) # (syringepump id, d_volume, rate)
time.sleep(sleep_time)
PS200_WF.Set_Home_SyringePump(1) # (syringepump id)
time.sleep(sleep_time)
PS200_WF.Set_Home_FractionCollector()
time.sleep(sleep_time)

time.sleep(sleep_time)
d=time.time()-s
#print(d)

## Initialize & Run Potentiostat

In [ ]:
sleep_time=1.5
PS200_WF.call_Initialize_SP200_API(SP200_config_params)
time.sleep(sleep_time)
PS200_WF.call_Connect_SP200()
time.sleep(sleep_time)
PS200_WF.call_Load_Firmware_SP200()
time.sleep(sleep_time)
PS200_WF.call_Initialize_CV_Tech_SP200(SP200_Technique_params)
time.sleep(sleep_time)
PS200_WF.call_Load_Technique_SP200()
time.sleep(sleep_time)
PS200_WF.call_Start_Channel_SP200()
time.sleep(sleep_time)


# profiles_via_data_channel=True  # Measurements sent via data channel 
# # CV_data_file=PS200_WF.call_Get_Measurement_SP200(testing_data_path,profiles_via_data_channel) # over one drive
#  CV_data_file=PS200_WF.call_Get_Measurement_SP200(testing_data_path_server,profiles_via_data_channel) # over file mount


# # profiles_via_data_channel=False # Measurements sent via control channel
CV_data_file=PS200_WF.call_Get_Measurement_SP200(testing_data_path,profiles_via_data_channel)



## I-V Profile Fitting & Classification

In [ ]:
save_file_flag=True
show_IV_plot=True

current_measurement_profile,measurement_status=call_analyze_CV_profile(CV_data_file,v_probe,testing_data_path,EoT_Classifier_Path, EoT_Classifier,save_file_flag,show_IV_plot)
print(f" CV Measurements of {os.path.basename(current_measurement_profile)} is \
   {('Normal') if measurement_status else ('Invalid')}")


## Terminate Electrochemistry Workflow

In [ ]:
PS200_WF.call_Stop_Channel_SP200()
PS200_WF.call_Disconnect_SP200()
#PS200_WF.call_Exit_JKem_API()
PS200_WF.call_Shutdown_SP200()